In [40]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [41]:
from gensim.models import KeyedVectors

In [42]:
ft=KeyedVectors.load("/content/gdrive/MyDrive/FYP/CAA_NRC/FastText/ft")

In [43]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/gdrive/MyDrive/FYP/CAA_NRC/CAA_NRC-preprocessed.csv")
y=pd.get_dummies(df["label"])


In [44]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

61


In [45]:
import tensorflow as tf
maxlen=maxi
max_words=len(ft.wv.vocab.keys())
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])
X

array([[   0,    0,    0, ...,    7,   55,   69],
       [   0,    0,    0, ...,   77,  139,  871],
       [   0,    0,    0, ...,   76,   81, 1784],
       ...,
       [   0,    0,    0, ...,   48,   18,    3],
       [   0,    0,    0, ...,  223,   80,   90],
       [   0,    0,    0, ...,  315,  754,   50]], dtype=int32)

In [46]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
for word,i in tokenizer.word_index.items():
  
  if word in ft and i < max_words:
    embed_vector=ft[word]
    
  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [47]:
embed_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.1540215 ,  0.27959114, -0.63524926, ..., -0.36897102,
        -0.23944433,  0.64169711],
       [ 0.3075901 ,  0.44092676, -1.21600664, ..., -0.59981263,
        -0.41611618,  1.15348899],
       ...,
       [-0.00129198, -0.03178301, -0.07373209, ..., -0.11604574,
        -0.03050946,  0.10048701],
       [ 0.02209803,  0.00965601, -0.08019561, ..., -0.11493652,
        -0.0358474 ,  0.10602047],
       [-0.01879814, -0.03643889, -0.16464797, ..., -0.18298186,
        -0.04111321,  0.20429862]])

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [49]:
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout,Embedding,GRU,Dense,Flatten
model=Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=maxi,embeddings_initializer=Constant(embed_matrix)))
model.add(Flatten()) # loss stucks at about 
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(3,activation='softmax'))

In [50]:
import time
start = time.time()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)
stop = time.time()
print(f"Training time: {stop - start}s")

Epoch 1/100
37/37 [==============================] - 1s 4ms/step - loss: 1.1520 - accuracy: 0.3892
Epoch 2/100
37/37 [==============================] - 0s 4ms/step - loss: 1.0659 - accuracy: 0.4413
Epoch 3/100
37/37 [==============================] - 0s 4ms/step - loss: 1.0512 - accuracy: 0.4409
Epoch 4/100
37/37 [==============================] - 0s 4ms/step - loss: 1.0434 - accuracy: 0.4383
Epoch 5/100
37/37 [==============================] - 0s 4ms/step - loss: 1.0274 - accuracy: 0.4430
Epoch 6/100
37/37 [==============================] - 0s 4ms/step - loss: 1.0250 - accuracy: 0.4503
Epoch 7/100
37/37 [==============================] - 0s 4ms/step - loss: 1.0188 - accuracy: 0.4520
Epoch 8/100
37/37 [==============================] - 0s 4ms/step - loss: 1.0170 - accuracy: 0.4598
Epoch 9/100
37/37 [==============================] - 0s 4ms/step - loss: 1.0362 - accuracy: 0.4624
Epoch 10/100
37/37 [==============================] - 0s 4ms/step - loss: 1.0077 - accuracy: 0.4662
Epoch 11/

In [51]:
#Metric calculation function
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))

In [52]:
import numpy as np
y_pred=model.predict(X_test)
y_test=y_test.to_numpy()
l=[np.argmax(i) for i in y_pred]
M=[np.argmax(i) for i in y_test]
display_metrics(M,l)

0.6572164948453608
0.6433578747236864
0.6618010158858747
0.6328788445684057
